## Extract tiles from a multiplexed Thunder experiment

The goal of this script is to resave individual tiles from a lif file as tiffs to be compatible with assembly using ASHLAR algorithm.

The tiles will be saved in a format:

WELL_r_ROUND_ch_CHANNEL_TILE

Depending on the naming convention you may need to adjust name parsing to get the right images from the lif file and clean the string for saving.

In [1]:
import os
from readlif.reader import LifFile
from tifffile import imwrite

from thunder_functions import create_tiles_99

In [2]:
# full path to the lif file
file_path = r''

# path to the folder where the tiff files will be saved
dir_save = r''

# expected size of a single frame
frame_size = 2048

In [3]:
# open lif image
image_stack = LifFile(file_path)

In [ ]:
# check the content of the lif file

img_list = [[image.name, image.dims] for image in image_stack.get_iter_image()]
print(len(img_list))
img_list

In [12]:
# wells to be processed

well_list= list(set(['/'.join(image.name.split('/')[-2:]) for image in image_stack.get_iter_image() if not ('Merged' in image.name or '5X' in image.name)])) # get all wells
well_list = sorted(well_list, key=lambda x: (x.split('/')[0], int(x.split('/')[1])))
# well_list = []# example: ['D/3','D/9'] # specify manually
well_list

['D/1', 'D/2', 'D/3', 'D/4', 'D/5', 'D/6', 'D/9']

In [6]:
# rounds to be processed

round_list = list(set([image.name.split('_')[0] for image in image_stack.get_iter_image() if not (('Merged' in image.name) or ('5X' in image.name))])) # get all wells
round_list = sorted(round_list, key=lambda x: int(x[1:]))
#round_list = []# example: ['R1','R2'] # specify manually
round_list

['R1',
 'R3',
 'R4',
 'R5',
 'R6',
 'R7',
 'R8',
 'R9',
 'R10',
 'R11',
 'R12',
 'R13',
 'R14']

In [ ]:
for well in well_list:

    for r_int,r in enumerate(round_list):

        # print info about what is found
        img_list_names = [image.name for image in image_stack.get_iter_image() if ((r+'_' in image.name) and (well in image.name) and not('Merged' in image.name))]
        print(img_list_names)
        ch_list = [image.channels for image in image_stack.get_iter_image() if ((r+'_' in image.name) and (well in image.name) and not('Merged' in image.name))]
        print(ch_list)

        # get images
        img_list = [image for image in image_stack.get_iter_image() if ((r+'_' in image.name) and (well in image.name) and not('Merged' in image.name))]

        # extract tiles
        t_list = create_tiles_99(img_list, ch_list)

        # save tiles
        for ind,t in enumerate(t_list):

            for ch in range(sum(ch_list)):

                imwrite(os.path.join(dir_save,f"{well.replace('/','')}_r_{str(r[1:]).zfill(2)}_ch_{str(ch).zfill(2)}_{str(ind).zfill(2)}.tif"), t[ch,:,:].astype('uint16'))
